<a href="https://colab.research.google.com/github/shinjangwoon/TIL/blob/master/Original_Upgrade_Magic_formula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오리지널 및 업그레이드 마법공식

|버전|레벨|스타일|기대CAGR|종목개수|매수전략|
|:--:|:--:|:--:|:--:|:--:|:--:|
|**오리지널**|초급|밸류+퀄리티|10~15%|20 ~ 30개|- EV/EBITDA & ROC 순위 매김 - 통합 순위 작성|
|**업그레이드**|중급|밸류+퀄리티|15~20%|20 ~ 30개|- EV/EBITDA & GP/A 순위 매김 - 통합 순위 작성|

# Basic Setting

앞으로는 ```Fn Guide```의 웹 페이지에서 크롤링을 해옵니다. ```투자지표``` 페이지와 ```재무제표``` 페이지의 기본 URL은 아래와 같습니다.

In [ ]:
# Parsing URL
INDEX_URL = 'http://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701'
FS_URL = 'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'


더불어 새로운 크롤링 방법을 사용하기 위해 ```BeautifulSoup```과 ```urllib``` 모듈을 가져옵니다.

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib import request as rq
from tqdm import tqdm

예시 코드로 ```카카오```의 코드를 사용합니다.

In [ ]:
# example code
code = '035720' # Kakao

# Get EV/EBITDA

가장 먼저 ```EV/EBITDA``` 데이터를 크롤링 해옵니다. 이를 위해 ```URL```에 들어가 해당 ```html``` 코드를 읽어오고 ```BeautifulSoup```을 사용해 구조화시키는 작업을 합니다. 특별히 ```html.parser``` 기능을 사용해 ```index_soup``` 변수로 지정합니다.

In [ ]:
index_html = rq.urlopen(INDEX_URL % code).read()
index_soup = BeautifulSoup(index_html, 'html.parser')

크롤링 해온 모든 ```html``` 코드 중에서 우리가 필요한 ```EV/EBITDA``` 수치를 가리키는 ```tag```만 가져옵니다. 웹 페이지에 들어가 ```검사(Inspeciton)```을 눌러 해당 수치를 가리키는 ```tag```를 찾아냅니다.

```tag```를 찾을 때 고유한 ```id```를 찾는 것이 중요합니다. 해당 수치가 속한 ```표(table)``` 또는 ```행(row)```의 ```id``` 값을 ```find``` 함수를 사용해 찾은 후 특정 ```데이터 값(td)```을 ```find_all``` 함수를 사용해 모두 찾아냅니다. 이 리스트를 ```ev_cells``` 변수로 지정합니다.

In [ ]:
ev_cells = index_soup.find('tr', {'id':'p_grid1_14'}).find_all('td')

해당 리스트 중 우리가 필요한 가장 최신의 수치를 ```slicing```한 후 ```tag```를 제외한 ```문자열``` 값만 도출하기 위해 ```.string```을 사용합니다.

In [ ]:
ev = float(ev_cells[3].string)

# Get Gross Profit

```INDEX_URL```과 마찬가지로 ```FS_URL```을 구조화된 형태로 크롤링 해옵니다.

In [ ]:
fs_html = rq.urlopen(FS_URL % code).read()
fs_soup = BeautifulSoup(fs_html, 'html.parser')

이번에는 고유한 값을 가진 ```id```가 ```divSonikY```라는 ```div(division)```에 속해있으므로 ```find```를 통해 찾아냅니다. 이후 ```class```가 ```rwf```라는 값을 가진 ```tr(table row ; 행)```을 찾아냅니다.

In [ ]:
gp_cells = fs_soup.find('div', {'id':'divSonikY'}).find_all('tr', {'class':'rwf'})

해당 ```Tag``` 데이터 중 우리가 원하는 수치들만 가져오기 위해 ```for 반복문```을 사용해 비어있는 ```리스트```에 담습니다.

In [ ]:
gp_list = []

for cell in gp_cells[2]:
    if cell != '\n':
        gp_list.append(cell.string)

```','``` 가 포함된 문자열은 실수형으로 변환되지 않으므로 ```replace``` 함수를 사용해 ```','```를 제거한 후 실수형 데이터로 변환합니다.

In [ ]:
gp = float(gp_list[4].replace(',',''))

# Get Asset

```매출총이익```과 마찬가지로 ```자산``` 데이터 역시 같은 방법을 사용해 크롤링 합니다.

In [ ]:
asset_cells = fs_soup.find('div', {'id':'divDaechaY'}).find_all('tr', {'class':'rwf'})

asset_list = []

for cell in asset_cells[0]:
    if cell != '\n':
        asset_list.append(cell.string)

In [ ]:
asset = float(asset_list[4].replace(',',''))

# Get All Stock Code

```pandas```를 사용해 ```company.csv``` 파일을 읽어옵니다.

In [ ]:
company = pd.read_csv('company.csv')
# company.head()

각 회사의 종목코드를 리스트에 담습니다.

In [ ]:
code_list = company['종목코드'].dropna()

In [ ]:
code_list.head()

0    000155
1    00088K
2    010955
3    051915
4    071055
Name: 종목코드, dtype: object

In [ ]:
sample_code_list = code_list[700:800]

# Create Index DataFrame

사람이 2000개 넘는 종목 하나하나 모두 크롤링 해오지 않고 컴퓨터가 대신 수행할 수 있도록 ```함수```를 사용하겠습니다. 우리가 만드는 ```filteringDf```라는 함수는 크게 1) 지표 크롤링 2) 크롤링 데이터 저장 3) DataFrame 변환 4) 오류처리 의 네가지 기능을 수행합니다. 

In [ ]:
# 함수를 선언합니다.
def filteringDf(li):
    
    # 크롤링 해온 데이터를 저장할 비어있는 딕셔너리를 만듭니다.
    result = {}
    
    # 2000개 넘는 종목의 코드를 하나하나 반복합니다.
    for code in tqdm(li):
        
        # 오류 처리
        try:
            # get company name : company.csv에서 종목코드에 매칭되는 회사명을 가져옵니다.
            name = company[company['종목코드'] == code]['회사명'].values[0]

            # set URL : 크롤링에 필요한 URL을 코드가 바뀔 때마다 세팅합니다.
            index_html = rq.urlopen(INDEX_URL % code).read()
            index_soup = BeautifulSoup(index_html, 'html.parser')

            fs_html = rq.urlopen(FS_URL % code).read()
            fs_soup = BeautifulSoup(fs_html, 'html.parser')

            # get EV/EBITDA : 각 종목코드에 맞는 EV/EBITDA 데이터를 크롤링합니다.
            ev_cells = index_soup.find('tr', {'id':'p_grid1_14'}).find_all('td')
            ev = float(ev_cells[3].string)

            # get Gross_Profit : 각 종목코드에 맞는 Gross_Profit 데이터를 크롤링합니다.
            gp_cells = fs_soup.find('div', {'id':'divSonikY'}).find_all('tr', {'class':'rwf'})

            gp_list = []

            for cell in gp_cells[2]:
                if cell != '\n':
                    gp_list.append(cell.string)

            gp = float(gp_list[4].replace(',',''))

            # get Asset : 각 종목코드에 맞는 Asset 데이터를 크롤링합니다.
            asset_cells = fs_soup.find('div', {'id':'divDaechaY'}).find_all('tr', {'class':'rwf'})

            asset_list = []

            for cell in asset_cells[0]:
                if cell != '\n':
                    asset_list.append(cell.string)

            asset = float(asset_list[4].replace(',',''))
            
            # 코드가 바뀌기 전에 크롤링 한 데이터를 reuslt 딕셔너리에 저장합니다.
            result[name] = [code, ev, gp, asset]
        
        # 오류 처리 (발생 가능한 모든 오류를 처리합니다.)
        except (TypeError, IndexError, AttributeError, ValueError) as err :
            
            # 오류가 발생할 경우 pass로 넘어갑니다.
            pass
        
    # create DataFrame : 딕셔너리 형태로 되어있는 크롤링 데이터를 Pandas DataFrame으로 변환합니다.
    result = pd.DataFrame(result)
    
    # 우리가 보기 좋은 형태로 DataFrame을 전치(transpose) 합니다.
    result = result.transpose()
    
    # 각 칼럼의 이름을 명명합니다.
    column_names = ['Code', 'EV/EBITDA', ' G/P', 'Asset']
    result.columns = column_names
    
    # 최종적으로 우리가 만든 DataFrame을 반환/도출 합니다.
    return result

위에서 선언한 함수에 모든 종목의 코드가 담긴 ```code_list```를 넣어 실행해보겠습니다. 그리고 그 결과값을 ```result```라는 변수에 담도록 하겠습니다. 앞으로 우리는 ```result```를 사용해서 원하는 연산 또는 가공을 할 수 있습니다.

In [ ]:
result = filteringDf(sample_code_list)
result.head()

100%|██████████| 100/100 [01:42<00:00,  1.02s/it]


,Code,EV/EBITDA,G/P,Asset
신일산업,002700,45.03,380,1396
삼양식품,003230,7.53,1517,5505
NI스틸,008260,6.38,223,2730
대동공업,000490,9.17,1604,9452
오리온홀딩스,001800,4.93,7421,45316


```result```라는 변수에 담은 ```원본 데이터```는 그대로 놔두고 ```복사본```을 만듭니다. 원본 데이터를 가공해 작업하다가 다시 원본 데이터를 사용하고 싶을 경우 오랜 시간 크롤링 해와야 하는 단점이 있습니다. 앞으로 우리는 ```.copy()```를 사용해 ```복사본```을 만들고 이를 가공하도록 하겠습니다.

In [ ]:
copy_df = result.copy()
copy_df.head()

,Code,EV/EBITDA,G/P,Asset
신일산업,002700,45.03,35,895
삼양식품,003230,7.53,478,4959
NI스틸,008260,6.38,56,2656
대동공업,000490,9.17,520,9031
오리온홀딩스,001800,4.93,2504,44239


# Searching Stocks

우리에게 필요한 칼럼만 정렬해서 볼 수 있습니다.

In [ ]:
copy_df.columns = ['Code', 'EV/EBITDA', 'GP', 'Asset']
copy_df

,Code,EV/EBITDA,GP,Asset
신일산업,002700,45.03,35,895
삼양식품,003230,7.53,478,4959
NI스틸,008260,6.38,56,2656
대동공업,000490,9.17,520,9031
오리온홀딩스,001800,4.93,2504,44239
깨끗한나라,004540,9.52,384,5557
태원물산,001420,46.07,-0,380
코오롱,002020,8.72,1680,36034
삼성전자,005930,6.6,205185,3.57458e+06
대유플러스,000300,4.18,118,4017


필터링을 위해 필요한 ```GP/A``` 값이 없기 때문에 ```GP```를 ```Asset```으로 나눠 새로운 칼럼을 만듭니다.

In [ ]:
copy_df['GP/A'] = copy_df['GP'] / copy_df['Asset']

우리에게 필요한 값이 모두 세팅됐습니다.

In [ ]:
copy_df = copy_df[['Code', 'EV/EBITDA', 'GP/A']]

# Grading Score

지수가 좋은 순서대로 상위 30%, 중위 40%, 하위 30%로 나눈 뒤 각각 3점, 2점, 1점의 점수를 매겨보겠습니다. ```EV/EBITDA```와 ```GP/A``` 지표를 기준으로 모든 종목에 점수를 매긴 뒤 합산 점수를 토대로 상위 종목을 필터링 하는 것이 목표입니다.

이를 위해 상위, 중위, 하위의 범위를 설정하겠습니다.

In [ ]:
# 30% / 40% / 30%
# 3 / 2 / 1
high_range = int(len(copy_df) * 0.3)
middle_range = int(len(copy_df) * 0.7)


```EV/EBITDA```를 기준으로 각 종목별 점수를 매겨보겠습니다.
1. ```copy_df```를 ```EV/EBITDA```를 기준으로 ```sort_values``` 함수를 사용해 정렬합니다.
```python
copy_df.sort_values(by='EV/EBITDA')
```
2. 상위 30% 종목을 ```slicing```을 사용해 걸러낸 후 ```종목코드```만 도출합니다.
```python
copy_df.sort_values(by='EV/EBITDA')[:high_range]['Code']
```
3. ```.isin``` 함수를 사용해 우리가 뽑아낸 상위 30% 종목의 종목코드가 원본 ```copy_df``` 종목코드에 속해있는지 판별합니다.
```python
copy_df['Code'].isin(copy_df.sort_values(by='EV/EBITDA')[:high_range]['Code'])
```
4. ```True```로 나온 값들에게만 ```.loc``` 함수를 사용해 ```EV/EBITDA_Score```라는 새로움 칼럼을 만들고서 ```3.0```점을 부여합니다.
```python
copy_df.loc[copy_df['Code'].isin(copy_df.sort_values(by='EV/EBITDA')[:high_range]['Code']), 'EV/EBITDA_Score']
```
5. 중위 40%, 하위 30%에도 같은 방법으로 각각 ```2.0```점, ```1.0```점을 부여합니다.

In [ ]:
# EV/EBITDA Scoring
copy_df.loc[copy_df['Code'].isin(copy_df.sort_values(by='EV/EBITDA')[:high_range]['Code']), 'EV/EBITDA_Score'] = 3.0 
copy_df.loc[copy_df['Code'].isin(copy_df.sort_values(by='EV/EBITDA')[high_range:middle_range]['Code']), 'EV/EBITDA_Score'] = 2.0 
copy_df.loc[copy_df['Code'].isin(copy_df.sort_values(by='EV/EBITDA')[middle_range:]['Code']), 'EV/EBITDA_Score'] = 1.0

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

```GP/A```를 기준으로 각 종목별 점수를 매겨보겠습니다.
1. ```copy_df```를 ```GP/A```를 기준으로 ```sort_values``` 함수를 사용해 정렬합니다. 다만 이번에는 높은 숫자에서 내림차순으로 정렬합니다.
```python
copy_df.sort_values(by='GP/A', ascending=False)
```
2. 상위 30% 종목을 ```slicing```을 사용해 걸러낸 후 ```종목코드```만 도출합니다.
```python
copy_df.sort_values(by='GP/A', ascending=False)[:high_range]['Code']
```
3. ```.isin``` 함수를 사용해 우리가 뽑아낸 상위 30% 종목의 종목코드가 원본 ```copy_df``` 종목코드에 속해있는지 판별합니다.
```python
copy_df['Code'].isin(copy_df.sort_values(by='GP/A', ascending=False)[:high_range]['Code'])
```
4. ```True```로 나온 값들에게만 ```.loc``` 함수를 사용해 ```GP/A_Score```라는 새로움 칼럼을 만들고서 ```3.0```점을 부여합니다.
```python
copy_df.loc[copy_df['Code'].isin(copy_df.sort_values(by='GP/A', ascending=False)[:high_range]['Code']), 'GPA_Score']
```
5. 중위 40%, 하위 30%에도 같은 방법으로 각각 ```2.0```점, ```1.0```점을 부여합니다.

In [ ]:
# GP/A Scoring
copy_df.loc[copy_df['Code'].isin(copy_df.sort_values(by='GP/A', ascending=False)[:high_range]['Code']), 'GPA_Score'] = 3.0
copy_df.loc[copy_df['Code'].isin(copy_df.sort_values(by='GP/A', ascending=False)[high_range:middle_range]['Code']), 'GPA_Score'] = 2.0
copy_df.loc[copy_df['Code'].isin(copy_df.sort_values(by='GP/A', ascending=False)[middle_range:]['Code']), 'GPA_Score'] = 1.0

```EV/EBITDA_Score```와 ```GP/A_Score```를 합산한 ```Total_Score``` 값을 만들어냅니다.

In [ ]:
copy_df['Total_Score'] = copy_df['EV/EBITDA_Score'] + copy_df['GPA_Score']

```Total_Score```가 높은 종목부터 내림차순으로 정렬합니다.

In [ ]:
copy_df = copy_df.sort_values(by='Total_Score', ascending=False)

상위 30개 종목만 결과적으로 도출합니다.

In [ ]:
copy_df.head(30)

,Code,EV/EBITDA,GP/A,EV/EBITDA_Score,GPA_Score,Total_Score
매일유업,267980,5.71,0.146575,3.0,3.0,6.0
대상,001680,5.47,0.077075,3.0,3.0,6.0
진양산업,003780,4.68,0.0608229,3.0,3.0,6.0
아이에이치큐,003560,6.18,0.136816,3.0,3.0,6.0
삼성전자,005930,6.6,0.0574012,3.0,3.0,6.0
엔피씨,004250,2.91,0.0403534,3.0,2.0,5.0
한국유리공업,002000,10.62,0.0769872,2.0,3.0,5.0
대한제분,001130,4.09,0.047384,3.0,2.0,5.0
동일방직,001530,6.4,0.0530242,3.0,2.0,5.0
아모레퍼시픽그룹,002790,9.78,0.11617,2.0,3.0,5.0


In [ ]:
copy_df[['Code', 'Total_Score']].head(30)

,Code,Total_Score
매일유업,267980,6.0
대상,001680,6.0
진양산업,003780,6.0
아이에이치큐,003560,6.0
삼성전자,005930,6.0
엔피씨,004250,5.0
한국유리공업,002000,5.0
대한제분,001130,5.0
동일방직,001530,5.0
아모레퍼시픽그룹,002790,5.0
